In [16]:
import pandas as pd
import requests

# ECOMMERCE ANALYSIS

# EXTRACTION

In [17]:
urlEcommerce_DataSource = 'https://62892c727af826e39e69681c.mockapi.io/api/SalesETL/Products'
csvEcommerce__DataSource = pd.read_csv('C:/Academic Projects/Sales ETL Project/sales_products.csv')

In [18]:
response = requests.get(urlEcommerce_DataSource)
if(response.status_code==200):
    print('Ok')
else: 
    print('Error')

Ok


### Creating dataframes

In [19]:
url_dataframe = pd.DataFrame(response.json())
csv_dataframe = csvEcommerce__DataSource

In [20]:
url_dataframe.head()

,name,price,description,date,id,brand,nombre,NAME
0,Handcrafted Bronze Car,837.00,New range of formal shirts are designed keepin...,2022-05-22T16:33:56.666Z,1,NaN,NaN,NaN
1,Recycled Fresh Ball,906.00,The Football Is Good For Training And Recreati...,2022-05-21T22:35:17.730Z,2,NaN,NaN,NaN
2,Recycled Rubber Bike,547.00,The automobile layout consists of a front-engi...,2022-05-22T06:55:55.364Z,3,NaN,NaN,NaN
3,Rustic Wooden Computer,129.00,The Football Is Good For Training And Recreati...,2022-05-22T01:18:21.524Z,4,NaN,NaN,NaN
4,Sleek Wooden Chicken,58.00,Carbonite web goalkeeper gloves are ergonomica...,2022-05-22T18:38:39.850Z,5,NaN,NaN,NaN


In [21]:
csv_dataframe.head()

,name,price,description,date,id
0,Bacon,166.0,The Nagasaki Lander is the trademarked name of...,2021-10-26T19:00:24.925Z,1
1,Bike,19.0,Ergonomic executive chair upholstered in bonde...,2021-10-03T18:02:12.519Z,2
2,Towels,216.0,The slim & simple Maple Gaming Keyboard from D...,2022-03-02T21:42:06.848Z,3
3,Hat,464.0,The Nagasaki Lander is the trademarked name of...,2022-05-20T11:46:17.171Z,4
4,Keyboard,819.0,"New ABC 13 9370, 13.3, 5th Gen CoreA5-8250U, 8...",2021-09-02T04:01:05.365Z,5


### Ordering columns

In [22]:
default_column_order = ['id', 'name', 'nombre', 'NAME', 'description', 'price', 'date', 'brand']
url_dataframe = url_dataframe.reindex(columns= default_column_order)

In [23]:
default_column_order = ['id', 'name', 'description', 'price', 'date']
csv_dataframe = csv_dataframe.reindex(columns= default_column_order)

### Concatenating Dataframes

In [24]:
Ecommerce_dataframe = pd.concat([url_dataframe, csv_dataframe],ignore_index=True)
print('url_dataframe rows & columns:' + str(url_dataframe.shape))
print('csv_dataframe rows & columns:' + str(csv_dataframe.shape))
print('Ecommerce_dataframe rows & columns:' + str(Ecommerce_dataframe.shape))
Ecommerce_dataframe.tail(1)

url_dataframe rows & columns:(62, 8)
csv_dataframe rows & columns:(10, 5)
Ecommerce_dataframe rows & columns:(72, 8)


,id,name,nombre,NAME,description,price,date,brand
71,10,Bike,NaN,NaN,The Football Is Good For Training And Recreati...,504.0,2021-11-17T05:18:05.217Z,NaN


#  TRANSFORM

## Removing Unnecessary Columns